In [96]:
import feedparser
import google.generativeai as genai
import re
import json

In [97]:
rss_url = 'https://www.ekonomidunya.com/rss_ekonomi_1.xml'

feed_eco = feedparser.parse(rss_url)

for entry in feed_eco.entries:
    print(f"Published: {entry.published}")
    print(f"Title: {entry.title}")
    print(f"Summary: {entry.summary}")
    print(f"Content: {entry.content[0].value}")
    print("-" * 80)

Published: Thu, 23 May 2024 14:01:56 +0300
Title: BES ve OKS'de toplam fon büyüklüğü ilk kez 1 trilyon lirayı aştı
Summary: Bireysel emeklilik sistemi (BES) ve otomatik katılım sisteminde (OKS) devlet katkısı dahil toplam fon büyüklüğü kurulduğu günden bu yana ilk defa 1 trilyon lirayı aşarken, katılımcı sayısı 16,4 milyona yaklaştı.
Content: Bireysel emeklilik ve otomatik katılım sisteminde toplam fon büyüklüğü 1 trilyon lirayı geçerken, katılımcı sayısı 16,4 milyona yaklaştı

Türkiye Sigorta Genel Müdürü Taha Çakmak:
"Bireysel emeklilik sistemi gerek sektör gerekse ülke ekonomisi için büyüyen bir değer"
AgeSA Hayat ve Emeklilik Genel Müdürü Fırat Kuruca:

"1 trilyon lirayı aşan büyüklük, ekonomiye yönlendirilen uzun vadeli yatırım miktarını gösteriyor"

Bireysel emeklilik sistemi (BES) ve otomatik katılım sisteminde (OKS) devlet katkısı dahil toplam fon büyüklüğü kurulduğu günden bu yana ilk defa 1 trilyon lirayı aşarken, katılımcı sayısı 16,4 milyona yaklaştı.

Bireysel emeklilik si

In [98]:
def feed_to_list(feed, list):
    for entry in feed.entries:
        entry_str = f"Published: {entry.published} \n" + f"Title: {entry.title} \n" + f"Summary: {entry.summary} \n" + f"Content: {entry.content[0].value}"
        list.append(entry_str)
    return list

eco_list = []
eco_list = feed_to_list(feed_eco, eco_list)
eco_list = list(reversed(eco_list))

In [100]:
import pandas as pd
df = pd.read_csv('stock_data_daily.csv')
df.Date = pd.to_datetime(df.Date)
df = df.set_index('Date')
df.tail()

,Open_XU100,High_XU100,Low_XU100,Close_XU100,Adj Close_XU100,Volume_XU100,Open_AEFES.IS,High_AEFES.IS,Low_AEFES.IS,Close_AEFES.IS,...,Adj Close_YYLGD.IS,Volume_YYLGD.IS,score_YYLGD.IS,Open_ZOREN.IS,High_ZOREN.IS,Low_ZOREN.IS,Close_ZOREN.IS,Adj Close_ZOREN.IS,Volume_ZOREN.IS,score_ZOREN.IS
Date,,,,,,,,,,,,,,,,,,,,,
2024-05-15,10171.099609,10241.000000,10121.700195,10158.599609,10158.599609,2379755600,197.000000,199.699997,192.600006,194.399994,...,13.05,9098754.0,50.0,5.92,5.98,5.81,5.85,5.85,143404679.0,50.0
2024-05-16,10189.099609,10320.000000,10145.099609,10320.000000,10320.000000,2964017900,194.399994,208.000000,194.399994,208.000000,...,13.17,5631192.0,50.0,5.88,6.12,5.85,6.07,6.07,353584533.0,50.0
2024-05-17,10369.900391,10652.299805,10357.000000,10643.599609,10643.599609,3252227500,208.000000,211.100006,206.000000,208.199997,...,13.19,7259318.0,50.0,6.09,6.20,6.07,6.09,6.09,179759508.0,50.0
2024-05-20,10660.799805,10775.000000,10633.900391,10739.599609,10739.599609,3307518800,208.600006,212.000000,205.500000,208.100006,...,13.50,15191617.0,50.0,6.10,6.34,6.09,6.31,6.31,230927389.0,50.0
2024-05-21,10767.339844,11088.009766,10751.980469,10943.190430,10943.190430,0,208.399994,212.399994,206.800003,209.000000,...,13.53,15567320.0,50.0,6.35,6.44,6.27,6.36,6.36,228724554.0,50.0


In [101]:
last_day_row = df.tail(1)
last_day_row_index = df.tail(1).index
score_columns = [col for col in df.columns if col.startswith('score_')]
last_day = last_day_row[score_columns]

In [102]:
keys = last_day.columns.to_list()
values = last_day.values.reshape((len(last_day.columns.to_list()), ))
result_dict = dict(zip(keys, values))
result_dict

{'score_AEFES.IS': 50.0,
 'score_AGROT.IS': 50.0,
 'score_AHGAZ.IS': 50.0,
 'score_AKBNK.IS': 50.0,
 'score_AKCNS.IS': 50.0,
 'score_AKFGY.IS': 50.0,
 'score_AKFYE.IS': 50.0,
 'score_AKSA.IS': 50.0,
 'score_AKSEN.IS': 50.0,
 'score_ALARK.IS': 50.0,
 'score_ALBRK.IS': 50.0,
 'score_ALFAS.IS': 50.0,
 'score_ANSGR.IS': 50.0,
 'score_ARCLK.IS': 50.0,
 'score_ASELS.IS': 50.0,
 'score_ASTOR.IS': 50.0,
 'score_BERA.IS': 50.0,
 'score_BFREN.IS': 50.0,
 'score_BIENY.IS': 50.0,
 'score_BIMAS.IS': 50.0,
 'score_BIOEN.IS': 50.0,
 'score_BOBET.IS': 50.0,
 'score_BRSAN.IS': 50.0,
 'score_BRYAT.IS': 50.0,
 'score_BTCIM.IS': 50.0,
 'score_CANTE.IS': 50.0,
 'score_CCOLA.IS': 50.0,
 'score_CIMSA.IS': 50.0,
 'score_CWENE.IS': 50.0,
 'score_DOAS.IS': 50.0,
 'score_DOHOL.IS': 50.0,
 'score_ECILC.IS': 50.0,
 'score_EKGYO.IS': 50.0,
 'score_ENERY.IS': 50.0,
 'score_ENJSA.IS': 50.0,
 'score_ENKAI.IS': 50.0,
 'score_EREGL.IS': 50.0,
 'score_EUPWR.IS': 50.0,
 'score_EUREN.IS': 50.0,
 'score_FROTO.IS': 50.0,
 's

In [83]:
system_str = "As a Market Analyst utilizing the OpenAI API key, your role involves analyzing daily economic news and its potential impact on specific stocks listed on Borsa Istanbul (BIST). \
      You are tasked with evaluating the relevance of news articles to the current scores of given stocks and adjusting their scores accordingly. Here's how your role operates: \
    Daily Task Overview: \
        Read and analyze daily economic news articles. \
        Identify any information that could potentially influence the prices of specified stocks listed on Borsa Istanbul. \
    Stock Score System: \
        Each stock has an initial score of 50.0, representing a neutral stance. \
        Your analysis will adjust these scores based on the perceived impact of the news on the respective stocks. \
        If significant information is found that could affect a stock's price, you will update its score accordingly on a scale of 0 to 100. \
    Adjusting Stock Scores: \
        If the news suggests a potential impact on a stock's price, assess the degree of influence and adjust the stock's score accordingly. \
        Scores can be adjusted upwards or downwards based on the perceived impact. \
    No Impact Scenario: \
        If the analysis does not reveal any information likely to affect a stock's price, maintain the stock's score at its current level. \
    Scoring Criteria: \
        Consider various factors such as financial performance, market sentiment, industry trends, and any other relevant data when adjusting scores. \
        Use a holistic approach to determine the magnitude of score adjustments, ensuring accuracy and consistency. \
    Documentation and Reporting: \
        Document your analysis process, including key findings and the rationale behind score adjustments. \
        Provide regular reports summarizing the impact of news articles on stock scores and any notable market trends. \
By effectively analyzing economic news and adjusting stock scores accordingly, you play a crucial role in providing valuable insights for investors and stakeholders in the Borsa Istanbul market."


In [21]:
API_KEY = 'GEMINI_API_KEY'

genai.configure(api_key=API_KEY)
model = genai.GenerativeModel('gemini-1.5-pro-latest',
                              system_instruction=system_str)

In [22]:
model

genai.GenerativeModel(
    model_name='models/gemini-1.5-pro-latest',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction="As a Market Analyst utilizing the OpenAI API key, your role involves analyzing daily economic news and its potential impact on specific stocks listed on Borsa Istanbul (BIST).       You are tasked with evaluating the relevance of news articles to the current scores of given stocks and adjusting their scores accordingly. Here's how your role operates:     Daily Task Overview:         Read and analyze daily economic news articles.         Identify any information that could potentially influence the prices of specified stocks listed on Borsa Istanbul.     Stock Score System:         Each stock has an initial score of 50.0, representing a neutral stance.         Your analysis will adjust these scores based on the perceived impact of the news on the respective stocks.         If significant information is found that could affect a s

In [24]:
eco_list = eco_list[:10]

In [25]:
def str_to_score_dict(old_dict, res_text):
    pattern = r'\{.*\}'
    match = re.search(pattern, res_text)

    if match:
        dict_string = match.group()
        result_dict = json.loads(dict_string)
        remaining_string = res_text.replace(dict_string, '')
        return result_dict, remaining_string
    else:
        return old_dict, ""

In [26]:
def score(eco_list, result_dict):
    analyze_list = []
    list_length = len(eco_list)

    for i in range(0, list_length, 5):
        news = ""
        chunk = eco_list[i:i+5]
        for new in chunk:
            news = news + new + "\n\n"
        str_dict = json.dumps(result_dict)
        response = model.generate_content(str_dict + '\n' + news)
        result_dict, analyze = str_to_score_dict(result_dict, response.text)
        analyze_list.append(analyze)
        
    return result_dict, analyze_list

In [27]:
new_result_dict, analyze_document = score(eco_list, result_dict)

In [116]:
for analysis in analyze_document:
    print(analysis)

```json

```

**Analysis:**

* **ALARK.IS (Alarko Holding):** Increased score to 65.0. The news about Alarko Holding's joint venture with Gotion Group for energy storage systems and lithium-ion battery production is highly positive. This strategic move aligns with the growing demand for renewable energy solutions and positions the company favorably in the market. 
* **TTRAK.IS (TürkTraktör):** Increased score to 60.0.  TürkTraktör reported strong financial results for the first quarter of 2024, with a significant net profit and a leading market share in the Turkish tractor market. This positive performance reflects the company's solid market position and growth potential.
* **YKBNK.IS (Yapı Kredi Bankası):** Increased score to 60.0.  Although Koç Holding stated that there are no concrete developments yet regarding the sale of its stake in Yapı Kredi Bankası, the news itself and the ongoing preliminary discussions have created positive sentiment around the stock, leading to a surge in i

: 

In [89]:
keys1 = last_day_row.columns.to_list()
values1 = last_day_row.values.reshape((len(last_day_row.columns.to_list()), ))
dict_final = dict(zip(keys1, values1))
dict_final

{'Open_XU100': 10767.33984375,
 'High_XU100': 11088.009765625,
 'Low_XU100': 10751.98046875,
 'Close_XU100': 10943.1904296875,
 'Adj Close_XU100': 10943.1904296875,
 'Volume_XU100': 0.0,
 'Open_AEFES.IS': 208.3999938964844,
 'High_AEFES.IS': 212.3999938964844,
 'Low_AEFES.IS': 206.8000030517578,
 'Close_AEFES.IS': 209.0,
 'Adj Close_AEFES.IS': 209.0,
 'Volume_AEFES.IS': 1817830.0,
 'score_AEFES.IS': 50.0,
 'Open_AGROT.IS': 30.899999618530277,
 'High_AGROT.IS': 31.780000686645508,
 'Low_AGROT.IS': 28.34000015258789,
 'Close_AGROT.IS': 28.780000686645508,
 'Adj Close_AGROT.IS': 28.780000686645508,
 'Volume_AGROT.IS': 33444435.0,
 'score_AGROT.IS': 50.0,
 'Open_AHGAZ.IS': 15.600000381469728,
 'High_AHGAZ.IS': 15.760000228881836,
 'Low_AHGAZ.IS': 14.9399995803833,
 'Close_AHGAZ.IS': 15.6899995803833,
 'Adj Close_AHGAZ.IS': 15.6899995803833,
 'Volume_AHGAZ.IS': 13517395.0,
 'score_AHGAZ.IS': 50.0,
 'Open_AKBNK.IS': 64.5999984741211,
 'High_AKBNK.IS': 70.75,
 'Low_AKBNK.IS': 64.4499969482421

In [90]:
def replace_dict_values(dict1, dict2):
  updated_dict = {key: dict2.get(key, value) for key, value in dict1.items()}
  return updated_dict

In [91]:
dict_final = replace_dict_values(dict_final, new_result_dict)
dict_final

{'Open_XU100': 10767.33984375,
 'High_XU100': 11088.009765625,
 'Low_XU100': 10751.98046875,
 'Close_XU100': 10943.1904296875,
 'Adj Close_XU100': 10943.1904296875,
 'Volume_XU100': 0.0,
 'Open_AEFES.IS': 208.3999938964844,
 'High_AEFES.IS': 212.3999938964844,
 'Low_AEFES.IS': 206.8000030517578,
 'Close_AEFES.IS': 209.0,
 'Adj Close_AEFES.IS': 209.0,
 'Volume_AEFES.IS': 1817830.0,
 'score_AEFES.IS': 50.0,
 'Open_AGROT.IS': 30.899999618530277,
 'High_AGROT.IS': 31.780000686645508,
 'Low_AGROT.IS': 28.34000015258789,
 'Close_AGROT.IS': 28.780000686645508,
 'Adj Close_AGROT.IS': 28.780000686645508,
 'Volume_AGROT.IS': 33444435.0,
 'score_AGROT.IS': 50.0,
 'Open_AHGAZ.IS': 15.600000381469728,
 'High_AHGAZ.IS': 15.760000228881836,
 'Low_AHGAZ.IS': 14.9399995803833,
 'Close_AHGAZ.IS': 15.6899995803833,
 'Adj Close_AHGAZ.IS': 15.6899995803833,
 'Volume_AHGAZ.IS': 13517395.0,
 'score_AHGAZ.IS': 50.0,
 'Open_AKBNK.IS': 64.5999984741211,
 'High_AKBNK.IS': 70.75,
 'Low_AKBNK.IS': 64.4499969482421

In [104]:
for key, value in dict_final.items():
  df.loc[last_day_row_index, key] = value



In [107]:
df.tail(1)[score_columns]

,score_AEFES.IS,score_AGROT.IS,score_AHGAZ.IS,score_AKBNK.IS,score_AKCNS.IS,score_AKFGY.IS,score_AKFYE.IS,score_AKSA.IS,score_AKSEN.IS,score_ALARK.IS,...,score_TUPRS.IS,score_TURSG.IS,score_ULKER.IS,score_VAKBN.IS,score_VESBE.IS,score_VESTL.IS,score_YEOTK.IS,score_YKBNK.IS,score_YYLGD.IS,score_ZOREN.IS
Date,,,,,,,,,,,,,,,,,,,,,
2024-05-21,50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0,65.0,...,50.0,50.0,50.0,50.0,50.0,50.0,50.0,60.0,50.0,50.0


In [108]:
df

,Open_XU100,High_XU100,Low_XU100,Close_XU100,Adj Close_XU100,Volume_XU100,Open_AEFES.IS,High_AEFES.IS,Low_AEFES.IS,Close_AEFES.IS,...,Adj Close_YYLGD.IS,Volume_YYLGD.IS,score_YYLGD.IS,Open_ZOREN.IS,High_ZOREN.IS,Low_ZOREN.IS,Close_ZOREN.IS,Adj Close_ZOREN.IS,Volume_ZOREN.IS,score_ZOREN.IS
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-02,7542.299805,7634.100098,7542.299805,7624.299805,7624.299805,2397081700,137.000000,140.000000,135.199997,138.000000,...,12.88,12438597.0,50.0,4.26,4.41,4.22,4.37,4.37,111809201.0,50.0
2024-01-03,7624.600098,7624.600098,7411.799805,7412.000000,7412.000000,2169646100,138.000000,138.199997,131.899994,131.899994,...,12.18,11671190.0,50.0,4.36,4.37,4.17,4.17,4.17,104408356.0,50.0
2024-01-04,7421.899902,7547.799805,7387.000000,7547.799805,7547.799805,2391591200,132.100006,132.800003,128.100006,131.300003,...,12.43,12559812.0,50.0,4.16,4.27,4.16,4.25,4.25,60182709.0,50.0
2024-01-05,7570.100098,7628.700195,7532.200195,7628.700195,7628.700195,2355101900,131.699997,132.899994,128.899994,131.399994,...,12.74,9836294.0,50.0,4.29,4.67,4.29,4.48,4.48,289683881.0,50.0
2024-01-08,7667.899902,7824.600098,7667.899902,7818.899902,7818.899902,2873781100,132.000000,133.000000,130.000000,130.300003,...,12.75,13399708.0,50.0,4.50,4.59,4.49,4.51,4.51,96531756.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-15,10171.099609,10241.000000,10121.700195,10158.599609,10158.599609,2379755600,197.000000,199.699997,192.600006,194.399994,...,13.05,9098754.0,50.0,5.92,5.98,5.81,5.85,5.85,143404679.0,50.0
2024-05-16,10189.099609,10320.000000,10145.099609,10320.000000,10320.000000,2964017900,194.399994,208.000000,194.399994,208.000000,...,13.17,5631192.0,50.0,5.88,6.12,5.85,6.07,6.07,353584533.0,50.0
2024-05-17,10369.900391,10652.299805,10357.000000,10643.599609,10643.599609,3252227500,208.000000,211.100006,206.000000,208.199997,...,13.19,7259318.0,50.0,6.09,6.20,6.07,6.09,6.09,179759508.0,50.0


In [109]:
df.to_csv('stock_data_daily.csv')